In [9]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from googletrans import Translator
from selenium.webdriver import Firefox
import time

browser = Firefox()

translator = Translator()

'''
Obs: 

-Se o selenium for mt lento para o trabalho, podemos tentar usar a lib Scrapy
-Algumas traduções são melhores feitas a mão

'''

#Site sendo testado - https://www.etf.com/AOR#overview

'\nObs: \n\n-Se o selenium for mt lento para o trabalho, podemos tentar usar a lib Scrapy\n-Algumas traduções são melhores feitas a mão\n\n'

In [4]:
#Pegando dados da planilha
data = pd.read_excel("Avenue Product List.xlsx", skiprows = 1)

#Seleciona os etfs
isEtf = data.Security == 'ETF'

#Armazena os tickers
tickers = data[isEtf].Symbol.values

In [10]:

#Endereço da página 
html = 'https://www.etf.com/IVV#overview'

#Entra no site com selenium
browser.get(html)

#Pega os textinhos através do xpath e traduz
'''Obs: Eu estava tendo dificuldade em encontrar os elementos, mas com o xpath eu consegui'''
fund_description = browser.find_element_by_xpath('//*[@id="overview"]/div[1]/section[1]/p').text
fund_description = translator.translate(fund_description, dest = 'pt').text
factSet_analytics = browser.find_element_by_xpath('/html/body/div[7]/section/div/div/div[2]/div[1]/div[1]/section[2]/p[2]').text
factSet_analytics = translator.translate(factSet_analytics, dest = 'pt').text

#Vai na tabelinha e pega os summary facts
summary_facts = []
for i in range(1,10):
    line = browser.find_element_by_xpath(f'//*[@id="fundSummaryData"]/section/div[{i}]').text
    label, content = line.split('\n')
    summary_facts.append((label, content))

#Pega os elementos da tabelinha portfolio data
element = browser.find_element_by_xpath('//*[@id="fundPortfolioData"]/section')
nElements_portfolio_data = element.find_elements_by_xpath('./child::*') #pega os elementos da tabelinha
portfolio_data = []
for i in range(1,len(nElements_portfolio_data)-1):
    line = browser.find_element_by_xpath(f'//*[@id="fundPortfolioData"]/section/div[{i}]').text
    label, content = line.split('\n')
    portfolio_data.append((label, content))

#Pega os maiores holdings do etf
top10_holdings = []
for i in range(1,11):
    line = browser.find_element_by_xpath(f'/html/body/div[7]/section/div/div/div[2]/div[4]/div[1]/div[2]/div/div[{i}]').text
    holding, weight = line.split('\n')
    top10_holdings.append((holding, weight))
# Criando dicionario com os dados
data2 = {
    'Fund Description' : fund_description,
    'Factsect Analytics' : factSet_analytics,
    'Summary Facts' : summary_facts,
    'Portfolio Data' : portfolio_data,
    '10 Holdings' : top10_holdings
}

In [134]:
'''Salvando em formato json'''
import json
with open('data.json', 'w', encoding='utf-8') as outfile:
    json.dump(data, outfile, ensure_ascii=False, indent=2)

In [166]:
browser.find_element_by_xpath(f'//*[@id="fundPortfolioData"]/section/div[{i}]').text

'Weighted Average Market Cap\n$355.23B'

In [11]:
data2

{'Fund Description': 'O IVV trava um índice ponderado pelo mercado de mercado de estoques de grande e midcap dos EUA selecionados pelo Comitê S & P.',
 'Factsect Analytics': 'IVV - Um dos vários ETFs que acompanham o índice S & P 500 - oferece excelente exposição de tampa grande.Apesar da percepção comum de que a S & P 500 proporciona uma exposição pura de boné de mercado à economia dos EUA, seu comitê pode exercer discrição para excluir empresas específicas.Além disso, IVV inclina-se ligeiramente menor do que o nosso benchmark, uma vez que atinge mais o espectro do mercado de mercado.Ainda assim, nossos analíticos mostram que essas distinções são essencialmente acadêmicas - o fundo oferece uma excelente cobertura.O IVV é estruturado como um Fundo de Ato de 1940, que comparado a outras estruturas, torna este favorável para os investidores de compra e espera que os dividendos possam ser reinvestidos quando pagos.O fundo também divulga as posições diárias ao contrário de outros.',
 'Summ